In [1]:
import ee
import geemap
import geopandas as gpd

import eeng

In [2]:
ee.Initialize()

In [3]:
asset = ee.FeatureCollection("users/ryangilberthamilton/randForestTrain_bq")
# define our region of interest
bellevilleBounds = asset.geometry().convexHull()
# define out time period
start, end = '2019-04-01', '2019-10-31'

In [4]:
# season filters
spring = ee.Filter.dayOfYear(135, 244)
summer = ee.Filter.dayOfYear(136, 244)

In [5]:
# Create our Setinel - 1 Inputs
from eeng.server.image_collection import Sentinel1Creator

s1_dv_asc = Sentinel1Creator().get_asc_dv_col(start_date=start, end_date=end, geometry=bellevilleBounds)
s1_dv_asc

In [6]:
# apply denoising and add Ratio Calculation
from eeng.server.filters import BoxCar
from eeng.server.calc import Ratio

boxcar = BoxCar(1)
ratio = Ratio()

s1_dv_asc = s1_dv_asc.denoise(boxcar).addCalculator(ratio)
s1_dv_asc

In [7]:
s1_spring_comp = s1_dv_asc.filter(spring).median()
s1_summer_comp = s1_dv_asc.filter(summer).median()
s1_input = ee.Image.cat(s1_spring_comp, s1_summer_comp)

In [8]:
# optical inputs
from eeng.server.image_collection import Sentinel2Creator
from eeng.server.calc import NDVI, SAVI, TasselCap
from eeng.server.cmasking import S2CloudlessAlgorithm
# create the base S2 SR collection
s2 = Sentinel2Creator()

s2sr = s2.get_sr_col(start_date=start, end_date=end, geometry=bellevilleBounds, cloud_cover=60)
s2cp = s2.get_cp_col(start_date=start, end_date=end, geometry=bellevilleBounds)
s2_cldls = s2.get_s2_cloudless_col(s2sr, s2cp)


In [9]:
# add the s2 Cloudless Algorithm
from eeng.server.cmasking import S2CloudlessAlgorithm

algo = S2CloudlessAlgorithm()
s2_cloudless = s2_cldls.addCloudMask(algo)

In [10]:
# add calculaors
from eeng.server.calc import NDVI, SAVI, TasselCap

NDVI = NDVI()
SAVI = SAVI()
TasselCap = TasselCap()

s2_clouldless = s2_cloudless.addCalculator(NDVI).addCalculator(SAVI).addCalculator(TasselCap)

In [11]:
# composit
s2_spring = s2_clouldless.filter(spring).median()
s2_summer = s2_clouldless.filter(summer).median()
s2_input = ee.Image.cat(s2_spring, s2_summer)

In [12]:
stack = ee.Image.cat(s1_input, s2_input)
stack.bandNames()

In [13]:
from eeng.server.tables import TrainingPoints
training = TrainingPoints(asset, 'B_Class')
training = training.addXCoordinate().addYCoordinate().addValues()
samples = training.generateSamples(stack)
# samples.points.first()

In [14]:
from eeng.server.models import RandomForestModel
from eeng.server.classifiers import RandomForestClassification

# create our model
rf_classifier = RandomForestClassification()

ValueError: model must be a RandomForestModel instance not <class 'str'>